<a href="https://colab.research.google.com/github/saravanamuthu1/sentiment_analysis/blob/main/naive_bayes_standard_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from collections import Counter
import math
import re



process the training data set


In [2]:
# we read the training data 
testfile = open('train-v2.tsv', 'r')
tweets=[]
label=[]
d_pos=0
d_neg=0
for line in testfile:
  value=line.rstrip("\n").split("\t")
  label.append(int(value[0]))
  tweets.append(value[1])
  if int(value[0]) == 0:
    d_neg+=1
  else:
    d_pos+=1

  

total probability for postiive documents and negative documents

 we have removed numbers from the data set, extra special characters. we also trimmed exrtra white spaces.

In [3]:
# pre-processing the training set
def clean_tweet(tweet):
    tweet = re.sub(r'[^\w\s]', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet=tweet.replace("USER","")
    tweet=tweet.strip()
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    return tweet


count the frequency of words as tuple pair as key in dictiionary 

In [4]:
# count the frequency of pair of (word,label)
def count_tweets(result,tweet,label):
  for i,j in zip(tweet,label):
    value=clean_tweet(i)
    value=value.split()
    for word in value:
      pair = (word,j)
      if pair in result:
          result[pair] += 1
      else:
          result[pair] = 1
  return result

In [5]:
result={}
count_tweets(result,tweets,label)



{('a', 0): 8614,
 ('sicrhau', 0): 9,
 ('bod', 0): 3352,
 ('mwy', 0): 548,
 ('o', 0): 8702,
 ('arian', 0): 60,
 ('poced', 0): 2,
 ('da', 0): 1314,
 ('ti', 0): 2348,
 ('nes', 0): 390,
 ('i', 0): 13334,
 ('fe', 0): 423,
 ('Efa', 0): 5,
 ('jst', 0): 53,
 ('never', 0): 39,
 ('again', 0): 40,
 ('creulondeb', 0): 1,
 ('ddyffryn', 0): 1,
 ('Aman', 0): 6,
 ('Parti', 1): 15,
 ('Dolig', 1): 105,
 ('da', 1): 2214,
 ('gyda', 1): 667,
 ('tim', 1): 209,
 ('swyddfa', 1): 43,
 ('canolog', 1): 1,
 ('Ty', 1): 29,
 ('Gwynfor', 1): 10,
 ('heno', 1): 1217,
 ('Pawb', 1): 75,
 ('yn', 1): 22078,
 ('brwdfrydig', 1): 4,
 ('ar', 1): 6595,
 ('gyfer', 1): 298,
 ('etholiad', 1): 18,
 ('Cynulliad', 1): 11,
 ('Cymru', 1): 417,
 ('blwyddyn', 1): 303,
 ('nesaf', 1): 134,
 ('yeaah', 0): 11,
 ('ma', 0): 4348,
 ('yn', 0): 24474,
 ('wir', 0): 212,
 ('oh', 0): 506,
 ('well', 0): 632,
 ('hahaha', 1): 488,
 ('idk', 1): 8,
 ('3am', 1): 3,
 ('oedd', 1): 746,
 ('y', 1): 8602,
 ('bws', 1): 25,
 ('ti', 1): 4337,
 ('dwim', 0): 977,


In [6]:
# dictionary raises a error when we search for invalid to avoaid this we put lookup function
def lookup(result,word,por):
  value= (word,por)
  if value in result.keys():
    final=result[value]
    return final
  else:
    return 0

In [7]:
# we calculate the lilikelihood for all the unique words in teh dictioanry 
def fit(freqs,label,total_doc):
    loglikelihood = {}
    logprior =0
    vocab = set([pair[0] for pair in freqs.keys()])
    print(len(vocab))
    V = len(vocab)
    N_pos = N_neg = 0
    for pair in freqs.keys():
      if int(pair[1]) == 1:
            N_pos += freqs[pair]
      else:
            N_neg += freqs[pair]
    D =total_doc
    d_pos = (len(list(filter(lambda x: x ==1, label))))
    d_neg = (len(list(filter(lambda x: x == 0, label))))
    logprior = np.log(d_pos) - np.log(d_neg)
    for word in vocab:
        freq_pos = lookup(freqs,word,1)
        freq_neg = lookup(freqs,word,0)
        # use one for smoothing
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)
        loglikelihood[word] = np.log(p_w_pos/p_w_neg)
    return logprior, loglikelihood

In [8]:
logprior, loglikelihood= fit(result,label,len(tweets)) 

85314


we add the probability of each word if word probaility is greater than 0 than we get conclude it as positive if it is less tha - we concude it as negative


In [9]:
#we add the probability of each word if word probaility si greater than 0 than we get conclude it as positive if it is less tha - we concude it as negative
def predict(tweet, logprior, loglikelihood):
  prediction=[]
  word_l=[]
  for i in  tweet:
    value=clean_tweet(i)
    word_l=value.split()
    p=0
    p=logprior
    for word in word_l:
      if word in loglikelihood:
        p += loglikelihood[word]
    if p > 0:
      prediction.append(1)
    else:
      prediction.append(0)
  return prediction

In [10]:
def evaluate():
  total =0
  correct_from_scratch = 0
  testfile1= open('test.tsv', 'r')
  label1=[]
  tweets1=[]
  for line in testfile1:
    value1=line.rstrip("\n").split("\t")
    label1.append(int(value1[0]))
    tweets1.append(value1[1])
    total += 1
  p = predict(tweets1, logprior, loglikelihood)
  for i in range(len(p)):
    if p[i] == label1[i]:
      correct_from_scratch+=1
  return (correct_from_scratch/total)
  

In [11]:
evaluate()

0.781